# MonkeyPox  Analysis

****Resently, monkeypox have been making the wave around the world. People are concern that it may be another outbreak. Confirmed cases have been reported in 36 countries till date.  This analysis shows the exploratory analysis of the monkeypox. Also, the analysis use a simple time series model to forecast for the daily cases.****

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Firstly, The data for the project were imported with the library that will be needed for Exploratory data analysis.**

In [2]:
#visualiztion library
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#data importation
data_1 = pd.read_csv('../input/monkeypox-dataset-daily-updated/Daily_Country_Wise_Confirmed_Cases.csv')
data_2 = pd.read_csv('../input/monkeypox-dataset-daily-updated/Monkey_Pox_Cases_Worldwide.csv')
data_3 = pd.read_csv('../input/monkeypox-dataset-daily-updated/Worldwide_Case_Detection_Timeline.csv')

print(f"Shapes of dataframes{data_1.shape, data_2.shape, data_3.shape}")

# Exploratory Data Analysis

In [3]:
#Gouping to check the daily reported cases
daily_case = data_3.groupby("Date_confirmation").count()

In [4]:
sns.lineplot(data=daily_case.Country)
plt.xticks(rotation=90)
plt.title("Monkey pox daily reported case worldwide")
plt.show()

In [5]:
#Top 10 country with the highest reported cases
sns.set(style='darkgrid')
data_3["Country"].value_counts()[:10].plot(kind="bar")
plt.show()


In [6]:
plt.figure(figsize = (15,8))
sns.countplot(x="Country", data=data_3, order = data_3["Country"].value_counts().index)
plt.xticks(rotation = 90)
plt.show()

In [7]:
#Checking gender distribution
plt.pie(data_3["Gender"].value_counts(), labels=data_3["Gender"].value_counts().index, autopct='%.0f%%')
plt.title("Gender")
plt.show()

In [8]:
plt.figure(figsize = (15,8))
sns.countplot(x="Country", data=data_3, hue = "Hospitalised (Y/N/NA)")
plt.xticks(rotation = 90)
plt.title("Cases Hospitalized by country")
plt.show()

In [9]:
plt.figure(figsize = (10,6))
sns.countplot(x="Gender", data=data_3, hue = "Hospitalised (Y/N/NA)")
plt.xticks(rotation = 90)
plt.show()

In [10]:
data_3.columns

In [11]:
sns.countplot(x='Travel_history (Y/N/NA)', data=data_3)
plt.show()

In [12]:
plt.figure(figsize=(10,8))
sns.countplot(y='Symptoms', data=data_3, order  = data_3['Symptoms'].value_counts().index)
plt.xticks(rotation = 90)
plt.show()

In [13]:
data_2.head()

In [14]:
data_1.head()

In [15]:
ts_data = data_3['Date_confirmation'].value_counts()
ts_data.columns = ['cases']
ts_data.sort_index(inplace=True)
ts_data = pd.DataFrame(ts_data)
ts_data.rename(columns = {'Date_confirmation':'cases'}, inplace = True)
ts_data

In [16]:
ts_data.plot()
plt.xticks(rotation = 45)
plt.xlabel("days")
plt.ylabel('daily total reported cases')
plt.title('')
plt.show()

In [17]:
#import libraries
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt
from pandas.plotting import autocorrelation_plot


In [18]:
autocorrelation_plot(ts_data)
plt.title("Autocorrelation Plot")
plt.show()

# Stationarity

We caculated the stationarity by using three days rollling.

In [19]:
rolling_mean = ts_data.rolling(3).mean()
rolling_std = ts_data.rolling(3).std()

**Plotting the time series data and the 3 days rolling mean and standard deviation**

In [20]:
plt.figure(figsize=(12,7))
plt.plot(ts_data, color="black",label="Original Passenger Data")
plt.plot(rolling_mean, color="blue", label="Rolling Mean daily case")
plt.plot(rolling_std, color="red", label = "Rolling Standard Deviation in daily case")
plt.title("Daily case Time Series, Rolling Mean, Standard Deviation")
plt.legend(loc="best")
plt.xticks(rotation=90)
plt.show()

# Augmented Dickey-Fuller test

In [21]:
from statsmodels.tsa.stattools import adfuller
ts_data =np.log(ts_data)

adft = adfuller(ts_data,autolag="AIC")

adft_df = pd.DataFrame({"Values":[adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']]  , "Metric":["Test Statistics","p-value","No. of lags used","Number of observations used", 
                                                        "critical value (1%)", "critical value (5%)", "critical value (10%)"]})
print(adft_df)

From the above output, it can be seen that our data is stationary at 90% confidence interval significant level. Eventhough the data shows increasing trend but the augmented Dickey-Fuller test shows it does not affect the stationarity of the data at 90%CF

# Autocorrelation

In [22]:
lag = {'One':1,'two':2,'three':3,'four':4,'Seven':7}
for j,i in lag.items():
    autocorrelation_lag1 = ts_data['cases'].autocorr(lag=i)
    print(f"{j} Month Lag: ", autocorrelation_lag1)

The autocorrelation output shows that the high correaltion between the data and lag 1 and lag 2 and he correlation decrease with increase in lag we can say there is long and shot time trend in the data after log. 

In [23]:
#Decomposition
from statsmodels.tsa.seasonal import seasonal_decompose
decompose = seasonal_decompose(ts_data['cases'],model='additive', period=3)
decompose.plot()
ax = plt.gca()
ax.axes.xaxis.set_ticklabels([])
plt.show()

In [26]:
pip install pmdarima

In [27]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
model = auto_arima(ts_data, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(ts_data)
forecast = model.predict(n_periods=len(ts_data))
forecast = pd.DataFrame(forecast,index = ts_data.index,columns=['Prediction'])

The best model for predicting this data is ARIMA(0,1,1). This model will be used to predict and we will check our prediction accuracy

In [28]:
rms = sqrt(mean_squared_error(ts_data,forecast))
print("RMSE: ", rms)

**There is no obvious trends in the data, and the stepwise AIC optimization  find meaningful autocorrelation  the time series. With the RMSE been low we say our model have a good forcast**

In [29]:
forecast

In [30]:
model.plot_diagnostics(figsize=(15,8))
plt.show()